# 第10章 深層学習によって実現できる画像処理・言語処理を知ろう(8-10節)
ここでは、深層学習を使った自然言語処理について学んでいきます。

Google Colaboratory上で実行する場合、ランタイムがGPUになっていることを確認して下さい

In [ ]:
#Colaboratory環境の設定
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MathProgramming/Chapter10

In [ ]:
#ライブラリの設定
!pip install -q -r ./requirements3.txt

## 10-9 Bertで文書の分類をしてみよう

In [ ]:
import pandas as pd
data_file='./spam.csv'
df = pd.read_csv('./spam.csv')
print(df["label"].value_counts())
df

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
tf.config.run_functions_eagerly(False)

#前処理をするモジュールの読み込み
bert_preprocess = hub.load("https://tfhub.dev/tensorflow/bert_en_cased_preprocess/2")

In [ ]:
test_preprocessed = bert_preprocess(["Hello World!"])
test_preprocessed

In [ ]:
#データを学習に７割り、テストに３割り使うように分ける
train_df = df[0: int(len(df)*0.7)]
test_df = df[int(len(df)*0.7):]

#前処理を行うモジュールで文字列の処理
X_train =  bert_preprocess(train_df["text"])
X_test = bert_preprocess(test_df["text"])

#ラベル(SpamとHam)をOnehot encoding
Y_train = pd.get_dummies(train_df["label"]).values.astype(np.float32)
Y_test = pd.get_dummies(test_df["label"]).values.astype(np.float32)

In [ ]:
#モデルの構築
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Dropout

#入力はinput_word_ids, input_mask, input_type_idsの３つ。
inputs = dict(
      input_word_ids=Input(shape=(None,), dtype=tf.int32),
      input_mask=Input(shape=(None,), dtype=tf.int32),
      input_type_ids=Input(shape=(None,), dtype=tf.int32))

#Tensorflow HubよりBertのモデルを読み込む
outputs = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1", trainable=True, name='bert_encoder')(inputs)
outputs = outputs["pooled_output"]
outputs = Dropout(0.1)(outputs)
#最終的な出力は２つ（SpamとHam)になるように全結合層を最後に付ける
outputs = Dense(2, activation="softmax", name='classifier')(outputs)
model = Model(inputs, outputs)

In [ ]:
from official.nlp import optimization
EPOCHS = 3
num_train_steps =  len(train_df.index) * EPOCHS
num_warmup_steps = int(0.1*num_train_steps)

#OptimizerとしてAdamWを利用
optimizer = optimization.create_optimizer(init_lr=0.00003,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])

#モデルの概要を出力
model.summary()

In [ ]:
#学習開始
hist = model.fit(X_train,Y_train,epochs=EPOCHS, validation_split=0.1)

In [ ]:
#学習に時間がかかるため、以下のコードのコメントアウトを外すことでモデルの重みの保存と読み込みが出来ます.
#１度保存しておき、10節以降をあとから実行する場合保存したモデルの重みを読み込むことで学習を再実行しなくて済みます

#学習した重みの保存
#model.save_weights('./saved_models/model_bert_weights')

In [ ]:
#重みの読み込み
#model.load_weights('./saved_models/model_bert_weights')

## 10-10 Bertを用いて分類した文章の評価をしてみよう



In [ ]:
#分類開始
pred = model.predict(X_test)

In [ ]:
pred_labels = np.array([np.argmax(p) for p in pred])
actual_labels = np.array([np.argmax(t) for t in Y_test])
tmp = actual_labels == pred_labels
tmp.sum()/len(tmp)

In [ ]:
#混同行列の表示

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cf_matrix = confusion_matrix(actual_labels, pred_labels)

c = sns.heatmap(cf_matrix, annot=True, fmt="d")

label_dict = {"ham": 0, "spam":1}
c.set(xticklabels=label_dict, yticklabels=label_dict)
plt.plot()

In [ ]:
#予測したテキスト
print("予測: " , pred_labels[0])
print(test_df.iloc[0]["text"])

In [ ]:
print("予測: " , pred_labels[3])
print(test_df.iloc[3]["text"])